In [23]:
# =====================================
# Imports and Environment Setup
# =====================================
import yfinance as yf
import pandas as pd
from sqlalchemy import create_engine
from dotenv import load_dotenv
import os
import urllib



In [25]:
# Load environment variables

DB_URL = os.getenv("DB_URL")  # optional if you want to use it
load_dotenv(override=True)

True

In [26]:
# SQL Server Connection
# =====================================
params = urllib.parse.quote_plus(
    "Driver={ODBC Driver 18 for SQL Server};"
    "Server=OLUWATOSIN,1433;"          # Named instance on fixed port
    "Database=MarketMVP;"
    "Trusted_Connection=yes;"
    "TrustServerCertificate=yes;"      # critical for avoiding SSL issues
)

engine = create_engine(f"mssql+pyodbc:///?odbc_connect={params}")

# Test connection
with engine.connect() as conn:
    print("✅ SQLAlchemy connected!")

# =====================================


✅ SQLAlchemy connected!


In [27]:
# Function to save DataFrame to SQL Server
# =====================================
def save_to_db(df: pd.DataFrame, table_name: str, engine=engine):
    """
    Save a DataFrame to SQL Server with proper column name handling.

    - Flattens MultiIndex or tuple column names.
    - Ensures SQL Server-compatible column names.
    """
    # Flatten MultiIndex or tuple column names
    if isinstance(df.columns, pd.MultiIndex):
        df.columns = ['_'.join(filter(None, map(str, col))) for col in df.columns]
    else:
        df.columns = [col[0] if isinstance(col, tuple) else col for col in df.columns]

    # Replace spaces and special characters
    df.columns = [col.replace(" ", "_").replace("-", "_") for col in df.columns]

    # Optional: lower-case columns (SQL Server is case-insensitive but consistent naming helps)
    df.columns = [col.lower() for col in df.columns]

    # Insert into SQL Server
    df.to_sql(table_name, engine, if_exists='append', index=False)
    print(f"✅ Data saved to table '{table_name}' successfully!")



In [28]:
# =====================================
# Example usage
# =====================================
if __name__ == "__main__":
    # Fetch stock data
    ticker = "AAPL"
    df = yf.download(ticker, start="2025-09-01", end="2025-09-17")
    df.reset_index(inplace=True)
    df['ticker'] = ticker  # add ticker column

    # Reorder columns to match SQL table
    df = df[['ticker', 'Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
    df.columns = ['ticker', 'date', 'open_price', 'high', 'low', 'close_price', 'volume']

    # Save to SQL
    save_to_db(df, "stock_data")

C:\Users\Admin\AppData\Local\Temp\ipykernel_1968\3360879948.py:7: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, start="2025-09-01", end="2025-09-17")
[*********************100%***********************]  1 of 1 completed

✅ Data saved to table 'stock_data' successfully!
